### zigbang 원룸 매물 데이터 수집
- 지역이름 > 매물정보 

In [1]:
import requests
import pandas as pd

In [2]:
# 1. 동이름 > 위도, 경도 
# url encoding : https://meyerweb.com/eric/tools/dencoder/
## https://meyerweb.com/eric/tools/dencoder/ (Decoding)

In [3]:
address = "마포구 성산동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸"
response = requests.get(url)
address_data = response.json()["items"][0]
lat, lng = address_data["lat"], address_data["lng"]
lat, lng

(37.56288146972656, 126.90928649902344)

In [4]:
# 2. 위도,경도 > geohash
# geohash2 설치

In [5]:
# !pip install geohash2

In [6]:
!pip install geohash2
import geohash2

In [7]:
# precision이 작아질수록 더 넓은 영역을 의미
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydjx'

In [8]:
# 3. geohash > item_ids

In [9]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0\
&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0\
&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [10]:
datas = response.json()["items"]
item_ids = [data["item_id"] for data in datas]
len(item_ids), item_ids[:5]

(929, [28920629, 29242318, 29271364, 29269875, 29153115])

In [11]:
# list conprehention
ls = [1, 2, 3, 4, 5]
result = []
for data in ls:
    result.append(data**2)
result

[1, 4, 9, 16, 25]

In [12]:
result = [data**2 for data in ls if data % 2]
result

[1, 9, 25]

In [13]:
# 4. item_ids > item info : POST

In [14]:
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": item_ids,
}
reponse = requests.post(url, params)
reponse

<Response [200]>

In [15]:
pd.options.display.max_columns = 30

In [16]:
datas = reponse.json()["items"]
df = pd.DataFrame(datas)
df = df[["item_id", "sales_type", "deposit", "rent", "size_m2",
         "building_floor", "floor", "title", "address1"]]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,building_floor,floor,title,address1
927,29271855,전세,28000,0,33.58,6,5,🔶🔶마포 저렴하고 전망 좋은 복층 쓰리룸🔶🔶,서울시 마포구 중동
928,29312091,전세,28000,0,36.00,6,5,💚초역세권💚 복층구조💚 막힘 없고 채광 GOOD💚,서울시 마포구 중동


In [17]:
# 함수로 만들기

In [27]:
def zigbang_oneroom(address):
    
    # 1. 동이름 > 위도, 경도
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸"
    response = requests.get(url)
    address_data = response.json()["items"][0]
    lat, lng = address_data["lat"], address_data["lng"]
    
    # 2. 위도,경도 > geohash
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # 3. geohash > item_ids
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0\
    &domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0\
    &sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    datas = response.json()["items"]
    item_ids = [data["item_id"] for data in datas]
    
    # 4. item_ids > item info : POST 
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": item_ids,
    }
    reponse = requests.post(url, params)
    datas = reponse.json()["items"]
    df = pd.DataFrame(datas)
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "building_floor", "floor", "title", "address1"]
    
    return df[columns]

In [29]:
address = "개포동"
df = zigbang_oneroom(address)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,building_floor,floor,title,address1
87,29206986,월세,2000,110,35.80,5,3,컨디션굿 가격굿 빠른입주가능,서울시 강남구 개포동
88,29090161,월세,30000,60,46.49,4,4,💎반전세 신축급 쓰리룸 거실아주넓은💎,서울시 강남구 개포동


In [34]:
df

,item_id,sales_type,deposit,rent,size_m2,building_floor,floor,title,address1
0,29307792,월세,1000,60,42.98,3,저,청계산입구 초 역세권 오피스텔 사업자용,서울시 서초구 신원동
1,29302254,전세,21000,0,59.50,5,저,청계산입구 초 역세권 귀한 전세매물,서울시 서초구 신원동
2,29036802,월세,2000,70,56.20,5,고,청계산입구 초 역세권 신축 오피스텔,서울시 서초구 신원동
3,29312374,전세,95000,0,105.79,3,1,💒테라스 소나무숲 view를 느끼세요! 마지막매물!!💒,서울시 서초구 내곡동
4,29234305,월세,20000,720,338.86,2,1,"◀집콕/정원+테라스▶러블리♥단독[4룸4화]신혼부부,작업실",서울시 서초구 염곡동
...,...,...,...,...,...,...,...,...,...
84,29265347,월세,70,70,39.67,6,고,💛실가격🧡No.1💚강남💜개포동 유일 단기풀옵션,서울시 강남구 개포동
85,29292587,월세,100,75,33.06,6,고,삼호물산 사거리 풀옵션 단기원룸,서울시 강남구 개포동
86,29304153,월세,21000,20,49.59,4,4,!!(급급급)개포동 신축급 가성비 최강의 2룸 반전세!!,서울시 강남구 개포동
87,29206986,월세,2000,110,35.80,5,3,컨디션굿 가격굿 빠른입주가능,서울시 강남구 개포동


In [40]:
filtered_df = df[df['address1'].str.contains("개포동")] ##true인 데이터만 필터링
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,building_floor,floor,title,address1
38,29206986,월세,2000,110,35.80,5,3,컨디션굿 가격굿 빠른입주가능,서울시 강남구 개포동
39,29090161,월세,30000,60,46.49,4,4,💎반전세 신축급 쓰리룸 거실아주넓은💎,서울시 강남구 개포동


In [ ]:
# 보증금 : 5000 이하, 렌트비 : 80 이하

In [49]:
# filtered_df.dtypes
result_df = filtered_df[
    (filtered_df['deposit'] <= 5000) & (filtered_df['rent'] <= 80)
]

result_df.tail()

,item_id,sales_type,deposit,rent,size_m2,building_floor,floor,title,address1
23,29291711,월세,3000,80,26.70,5,2,☎☎ 신축 풀옵션 넓으세요 + 테라스 ^^,서울시 강남구 개포동
25,29190666,월세,2000,45,23.14,5,옥탑방,☎☎ 나혼자 옥탑 !!! 리모델링한 풀옵션 원룸,서울시 강남구 개포동
34,29260020,월세,100,70,46.28,6,중,🛑깜놀주의🛑양재역 인근 단기방 최저가,서울시 강남구 개포동
35,29265347,월세,70,70,39.67,6,고,💛실가격🧡No.1💚강남💜개포동 유일 단기풀옵션,서울시 강남구 개포동
36,29292587,월세,100,75,33.06,6,고,삼호물산 사거리 풀옵션 단기원룸,서울시 강남구 개포동
